In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install selenium
!pip install selenium
!pip install bs4
# !pip install html_table_parser

In [ ]:
# install chromedriver
!apt-get update
!apt install chromium-chromedriver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 n

In [ ]:
# setting environmental varible to use chromedriver easily
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
# from html_table_parser import parser_functions as parser
import pandas as pd

from selenium.webdriver.support.select import Select
from urllib.request import urlopen

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


# Crawling stats of each team from each year

In [ ]:
URL1 = 'https://www.koreabaseball.com/Record/Team/Hitter/Basic1.aspx'
URL2 = 'https://www.koreabaseball.com/Record/Team/Pitcher/Basic1.aspx'
URL3 = 'https://www.koreabaseball.com/Record/Team/Defense/Basic.aspx'
URL4 = 'https://www.koreabaseball.com/Record/Team/Runner/Basic.aspx'


In [92]:
Data_list =[]
start_year = 2014 # inclusive
end_year = 2023 # inclusive

# data during start_year ~ end_year will be collected
for i in range(start_year, end_year+1):
    Data_list.append('KBO_Team_'+ str(i))

df_list = [] # team stats of each year will be stored here

for data in Data_list:
    data = pd.DataFrame()
    df_list.append(data)


In [93]:
# make a dictionary of Query variables in URL
URL_list = {"Hitter": "Basic1", "Hitter1": "Basic2",
            "Pitcher": "Basic1", "Pitcher1": "Basic2",
            "Defense":"Basic", "Runner":"Basic" }

# settings for GoogleColab
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # set Head-less
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# start crawling
for i in range(len(URL_list)): # 0~5
    # making a driver object
    driver = webdriver.Chrome(options=options)

    if (i==1)| (i==3):
        class_name = list(URL_list.keys())[i][:-1]
    else:
        class_name = list(URL_list.keys())[i]
    URL = f"https://www.koreabaseball.com/Record/Team/{class_name}/{list(URL_list.values())[i]}.aspx"


    # change URL
    driver.get(url = str(URL))
    # wait for 3 seconds to fully load the URL page
    time.sleep(3)

    # 0:2001, 1:2002, ..., 13:2014, ..., 22:2023
    for j in range(start_year - 2001, end_year - 2001 +1):
        # Select -> tag -> change year
        select_tag = Select(driver.find_element(By.ID,
                            "cphContents_cphContents_cphContents_ddlSeason_ddlSeason"))
        select_tag.select_by_index(j)
        year = 2001+j

        # delay 3 seconds to load HTML code
        time.sleep(3)

        # generate a dataframe from data of the chosen year
        # 선택한 년도의 표를 읽어서 데이터프레임으로 생성하기
        html = driver.page_source
        df = pd.read_html(html)[0]
        df = df[df["팀명"]!="합계"] # drop "total value" row
        # attach class_name("Hitter", "Pitcher", "Defense", "Runner") to some columns
        df.columns = [class_name+"_"+col_name if ((i not in [0,1]) and (col_name!="G"))
                        else col_name
                        for i, col_name in enumerate(df.columns)]
        df.insert(0, "Year", year) # attach year

        # Hitter -> merge with an empty dataframe
        idx = j - (start_year - 2001)
        if i == 0:
            df_list[idx] = pd.concat([df_list[idx], df], ignore_index=True)

        # Pitcher, Defense, Runner -> merge with existing Hitter data
        else :
            df_list[idx] = pd.merge(df_list[idx], df, on=['Year', '팀명'])

    # close the driver object
    driver.close()



<ipython-input-93-fdd0d51d888d>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'순위_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_list[idx] = pd.merge(df_list[idx], df, on=['Year', '팀명'])
<ipython-input-93-fdd0d51d888d>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'순위_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_list[idx] = pd.merge(df_list[idx], df, on=['Year', '팀명'])
<ipython-input-93-fdd0d51d888d>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'순위_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_list[idx] = pd.merge(df_list[idx], df, on=['Year', '팀명'])
<ipython-input-93-fdd0d51d888d>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'순위_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_list[idx] = pd.merge(df_list[idx], df, on=['Year', '팀명'])
<ipy

In [94]:
df_list[0]

,Year,순위_x,팀명,Hitter_AVG_x,G_x,Hitter_PA,Hitter_AB,Hitter_R,Hitter_H,Hitter_2B,...,Defense_CS,Defense_CS%,순위_y,G_y,Runner_SBA,Runner_SB,Runner_CS,Runner_SB%,Runner_OOB,Runner_PKO
0,2014,1,삼성,0.301,128,5152,4464,812,1345,238,...,44,30.6,1,128,207,161,46,77.8,60,10
1,2014,2,넥센,0.298,128,5163,4434,841,1323,275,...,44,27.2,7,128,144,100,44,69.4,54,13
2,2014,3,두산,0.293,128,5083,4427,687,1299,235,...,46,28.9,5,128,155,111,44,71.6,50,6
3,2014,4,SK,0.291,128,5082,4423,735,1285,245,...,51,33.1,3,128,185,139,46,75.1,50,11
4,2014,5,KIA,0.288,128,5023,4443,662,1279,246,...,64,28.3,4,128,184,121,63,65.8,59,7
5,2014,6,롯데,0.287,128,5194,4487,716,1288,252,...,37,30.6,9,128,104,63,41,60.6,49,2
6,2014,7,한화,0.283,128,5055,4417,619,1251,223,...,59,34.9,8,128,111,70,41,63.1,46,5
7,2014,8,NC,0.282,128,5031,4426,737,1249,228,...,43,31.4,2,128,201,154,47,76.6,44,7
8,2014,9,LG,0.279,128,5070,4349,668,1214,213,...,48,28.7,6,128,169,105,64,62.1,51,9


In [ ]:
# Remove duplicated data: ranking, # game, ERA, AVG
for i in range(len(df_list)):
    df_list[i] = df_list[i].drop(['순위_x', '순위_y', 'G_y', 'Pitcher_ERA_y', 'Hitter_AVG_y'], axis = 1)


In [130]:
total_df = pd.concat(df_list, ignore_index=True)
total_df.columns

Index(['Year', '팀명', 'Hitter_AVG_x', 'G_x', 'Hitter_PA', 'Hitter_AB',
       'Hitter_R', 'Hitter_H', 'Hitter_2B', 'Hitter_3B', 'Hitter_HR',
       'Hitter_TB', 'Hitter_RBI', 'Hitter_SAC', 'Hitter_SF', 'Hitter_BB',
       'Hitter_IBB', 'Hitter_HBP', 'Hitter_SO', 'Hitter_GDP', 'Hitter_SLG',
       'Hitter_OBP', 'Hitter_OPS', 'Hitter_MH', 'Hitter_RISP', 'Hitter_PH-BA',
       'Pitcher_ERA_x', 'Pitcher_W', 'Pitcher_L', 'Pitcher_SV', 'Pitcher_HLD',
       'Pitcher_WPCT', 'Pitcher_IP', 'Pitcher_H', 'Pitcher_HR', 'Pitcher_BB',
       'Pitcher_HBP', 'Pitcher_SO', 'Pitcher_R', 'Pitcher_ER', 'Pitcher_WHIP',
       'Pitcher_CG', 'Pitcher_SHO', 'Pitcher_QS', 'Pitcher_BSV', 'Pitcher_TBF',
       'Pitcher_NP', 'Pitcher_AVG', 'Pitcher_2B', 'Pitcher_3B', 'Pitcher_SAC',
       'Pitcher_SF', 'Pitcher_IBB', 'Pitcher_WP', 'Pitcher_BK', 'G_x',
       'Defense_E', 'Defense_PKO', 'Defense_PO', 'Defense_A', 'Defense_DP',
       'Defense_FPCT', 'Defense_PB', 'Defense_SB', 'Defense_CS', 'Defense_CS%',
       'R

In [133]:
total_df = total_df.drop(['G_x'], axis = 1)

In [134]:
total_df.columns

Index(['Year', '팀명', 'Hitter_AVG_x', 'Hitter_PA', 'Hitter_AB', 'Hitter_R',
       'Hitter_H', 'Hitter_2B', 'Hitter_3B', 'Hitter_HR', 'Hitter_TB',
       'Hitter_RBI', 'Hitter_SAC', 'Hitter_SF', 'Hitter_BB', 'Hitter_IBB',
       'Hitter_HBP', 'Hitter_SO', 'Hitter_GDP', 'Hitter_SLG', 'Hitter_OBP',
       'Hitter_OPS', 'Hitter_MH', 'Hitter_RISP', 'Hitter_PH-BA',
       'Pitcher_ERA_x', 'Pitcher_W', 'Pitcher_L', 'Pitcher_SV', 'Pitcher_HLD',
       'Pitcher_WPCT', 'Pitcher_IP', 'Pitcher_H', 'Pitcher_HR', 'Pitcher_BB',
       'Pitcher_HBP', 'Pitcher_SO', 'Pitcher_R', 'Pitcher_ER', 'Pitcher_WHIP',
       'Pitcher_CG', 'Pitcher_SHO', 'Pitcher_QS', 'Pitcher_BSV', 'Pitcher_TBF',
       'Pitcher_NP', 'Pitcher_AVG', 'Pitcher_2B', 'Pitcher_3B', 'Pitcher_SAC',
       'Pitcher_SF', 'Pitcher_IBB', 'Pitcher_WP', 'Pitcher_BK', 'Defense_E',
       'Defense_PKO', 'Defense_PO', 'Defense_A', 'Defense_DP', 'Defense_FPCT',
       'Defense_PB', 'Defense_SB', 'Defense_CS', 'Defense_CS%', 'Runner_SBA',
       'R

In [137]:
total_df.to_csv("/content/drive/MyDrive/Data_Science_Lab/KBO_Team_stat.csv",
                index=False)

# Crawling rankings of each team from each year

In [124]:
# start crawling

ranking_df = pd.DataFrame()

# making a driver object
driver = webdriver.Chrome(options=options)

URL = "https://www.koreabaseball.com/Record/TeamRank/TeamRank.aspx"

driver.get(url = str(URL))
# wait for 3 seconds to fully load the URL page
time.sleep(3)

# 0:2023, 1:2022, ..., 9:2014, ..., 41:1982
for j in range(0, end_year-start_year+1):
    # Select -> tag -> change year
    select_tag = Select(driver.find_element(By.ID,
                "cphContents_cphContents_cphContents_ddlYear"))
    select_tag.select_by_index(j)
    year = 2023-j

    # delay 3 seconds to load HTML code
    time.sleep(3)

    # generate a dataframe from data of the chosen year
    # 선택한 년도의 표를 읽어서 데이터프레임으로 생성하기
    html = driver.page_source
    df = pd.read_html(html)[0]
    df.insert(0, "Year", year) # attach year

    ranking_df = pd.concat([ranking_df, df], axis = 0, ignore_index = True)

# close the driver object
driver.close()

In [125]:
ranking_df

,Year,순위,팀명,경기,승,패,무,승률,게임차,최근10경기,연속,홈,방문
0,2023,1,LG,144,86,56,2,0.606,0.0,5승0무5패,1승,45-1-25,41-1-31
1,2023,2,KT,144,79,62,3,0.560,6.5,6승0무4패,3승,41-2-30,38-1-32
2,2023,3,SSG,144,76,65,3,0.539,9.5,8승0무2패,2승,39-2-32,37-1-33
3,2023,4,NC,144,75,67,2,0.528,11.0,5승0무5패,2패,41-1-29,34-1-38
4,2023,5,두산,144,74,68,2,0.521,12.0,4승0무6패,3패,37-1-35,37-1-33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2014,5,SK,128,61,65,2,0.484,17.5,6승1무3패,1패,31-2-31,30-0-34
95,2014,6,두산,128,59,68,1,0.465,20.0,6승0무4패,1승,29-0-35,30-1-33
96,2014,7,롯데,128,58,69,1,0.457,21.0,5승0무5패,1승,32-1-31,26-0-38
97,2014,8,KIA,128,54,74,0,0.422,25.5,3승0무7패,2승,26-0-38,28-0-36


In [139]:
ranking_df.to_csv("/content/drive/MyDrive/Data_Science_Lab/KBO_Team_rank.csv",
                    index=False)

# merge two dataframes

In [140]:
ranking_stat_df = ranking_df.merge(total_df, on = ['Year', '팀명'])
ranking_stat_df.columns

Index(['Year', '순위', '팀명', '경기', '승', '패', '무', '승률', '게임차', '최근10경기', '연속',
       '홈', '방문', 'Hitter_AVG_x', 'Hitter_PA', 'Hitter_AB', 'Hitter_R',
       'Hitter_H', 'Hitter_2B', 'Hitter_3B', 'Hitter_HR', 'Hitter_TB',
       'Hitter_RBI', 'Hitter_SAC', 'Hitter_SF', 'Hitter_BB', 'Hitter_IBB',
       'Hitter_HBP', 'Hitter_SO', 'Hitter_GDP', 'Hitter_SLG', 'Hitter_OBP',
       'Hitter_OPS', 'Hitter_MH', 'Hitter_RISP', 'Hitter_PH-BA',
       'Pitcher_ERA_x', 'Pitcher_W', 'Pitcher_L', 'Pitcher_SV', 'Pitcher_HLD',
       'Pitcher_WPCT', 'Pitcher_IP', 'Pitcher_H', 'Pitcher_HR', 'Pitcher_BB',
       'Pitcher_HBP', 'Pitcher_SO', 'Pitcher_R', 'Pitcher_ER', 'Pitcher_WHIP',
       'Pitcher_CG', 'Pitcher_SHO', 'Pitcher_QS', 'Pitcher_BSV', 'Pitcher_TBF',
       'Pitcher_NP', 'Pitcher_AVG', 'Pitcher_2B', 'Pitcher_3B', 'Pitcher_SAC',
       'Pitcher_SF', 'Pitcher_IBB', 'Pitcher_WP', 'Pitcher_BK', 'Defense_E',
       'Defense_PKO', 'Defense_PO', 'Defense_A', 'Defense_DP', 'Defense_FPCT',
       'Defen

In [143]:
ranking_stat_df.rename(columns = {'Hitter_AVG_x':'Hitter_AVG', 'Pitcher_ERA_x':'Pitcher_ERA'},
                       inplace = True)
ranking_stat_df.columns

Index(['Year', '순위', '팀명', '경기', '승', '패', '무', '승률', '게임차', '최근10경기', '연속',
       '홈', '방문', 'Hitter_AVG', 'Hitter_PA', 'Hitter_AB', 'Hitter_R',
       'Hitter_H', 'Hitter_2B', 'Hitter_3B', 'Hitter_HR', 'Hitter_TB',
       'Hitter_RBI', 'Hitter_SAC', 'Hitter_SF', 'Hitter_BB', 'Hitter_IBB',
       'Hitter_HBP', 'Hitter_SO', 'Hitter_GDP', 'Hitter_SLG', 'Hitter_OBP',
       'Hitter_OPS', 'Hitter_MH', 'Hitter_RISP', 'Hitter_PH-BA', 'Pitcher_ERA',
       'Pitcher_W', 'Pitcher_L', 'Pitcher_SV', 'Pitcher_HLD', 'Pitcher_WPCT',
       'Pitcher_IP', 'Pitcher_H', 'Pitcher_HR', 'Pitcher_BB', 'Pitcher_HBP',
       'Pitcher_SO', 'Pitcher_R', 'Pitcher_ER', 'Pitcher_WHIP', 'Pitcher_CG',
       'Pitcher_SHO', 'Pitcher_QS', 'Pitcher_BSV', 'Pitcher_TBF', 'Pitcher_NP',
       'Pitcher_AVG', 'Pitcher_2B', 'Pitcher_3B', 'Pitcher_SAC', 'Pitcher_SF',
       'Pitcher_IBB', 'Pitcher_WP', 'Pitcher_BK', 'Defense_E', 'Defense_PKO',
       'Defense_PO', 'Defense_A', 'Defense_DP', 'Defense_FPCT', 'Defense_PB',
   

In [144]:
ranking_stat_df

,Year,순위,팀명,경기,승,패,무,승률,게임차,최근10경기,...,Defense_PB,Defense_SB,Defense_CS,Defense_CS%,Runner_SBA,Runner_SB,Runner_CS,Runner_SB%,Runner_OOB,Runner_PKO
0,2023,1,LG,144,86,56,2,0.606,0.0,5승0무5패,...,10,115,30,20.7,267,166,101,62.2,78,15
1,2023,2,KT,144,79,62,3,0.560,6.5,6승0무4패,...,3,111,23,17.2,117,87,30,74.4,54,5
2,2023,3,SSG,144,76,65,3,0.539,9.5,8승0무2패,...,11,127,56,30.6,124,96,28,77.4,43,10
3,2023,4,NC,144,75,67,2,0.528,11.0,5승0무5패,...,7,96,42,30.4,157,111,46,70.7,53,8
4,2023,5,두산,144,74,68,2,0.521,12.0,4승0무6패,...,12,60,40,40.0,181,133,48,73.5,52,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2014,5,SK,128,61,65,2,0.484,17.5,6승1무3패,...,6,103,51,33.1,185,139,46,75.1,50,11
95,2014,6,두산,128,59,68,1,0.465,20.0,6승0무4패,...,7,113,46,28.9,155,111,44,71.6,50,6
96,2014,7,롯데,128,58,69,1,0.457,21.0,5승0무5패,...,11,84,37,30.6,104,63,41,60.6,49,2
97,2014,8,KIA,128,54,74,0,0.422,25.5,3승0무7패,...,7,162,64,28.3,184,121,63,65.8,59,7


In [145]:
ranking_stat_df.to_csv("/content/drive/MyDrive/Data_Science_Lab/KBO_Team_rank_stat.csv",
                    index=False)